In [ ]:
import pickle
from nltk import sent_tokenize, word_tokenize

In [ ]:
import re
def extract_features(sentence, index):
  return {
      'word':sentence[index],
      'is_first':index==0,
      'is_last':index ==len(sentence)-1,
      'prefix-1':sentence[index][0],
      'prefix-2':sentence[index][:2],
      'prefix-3':sentence[index][:3],
      'prefix-3':sentence[index][:4],
      'suffix-1':sentence[index][-1],
      'suffix-2':sentence[index][-2:],
      'suffix-3':sentence[index][-3:],
      'suffix-3':sentence[index][-4:],
      'prev_word':'' if index == 0 else sentence[index-1],
      'next_word':'' if index < len(sentence) else sentence[index+1],
      'has_hyphen': '-' in sentence[index],
      'is_numeric': sentence[index].isdigit(),
  }

In [ ]:
with open("./pos_crf_model.pkl", "rb") as f:
    model = pickle.load(f)
    print(type(model))

In [ ]:
def transform_to_dataset(sentence):
  tmp = []
  for index in range(len(sentence)):
      tmp.append(extract_features(sentence, index)),
  return tmp

In [ ]:
def transform_to_dataset(corpus: str):
    sentences = sent_tokenize(corpus)
    sentences = [word_tokenize(sentence) for sentence in sentences]
    X = []
    for sentence in sentences:
        tmp = []
        for i in range(len(sentence)):
            tmp.append(extract_features(sentence, i))
        X.append(tmp)
    return X

In [ ]:
input_txt = "இல்­லையா என்­பதை மக்கள் முன்­னி­லையில் குறிப்­பிட வேண்டும் .  அதேபோல் இம்­முறை வரவு–செலவு திட்டமானது மக்­க­ளுக்கு சிறி­த­ளவு நிவா­ரணம் அம்­சங்கள் சிலவும் உள்­ளன .  அதனால் இதனை தோற்கடிப்பதால் மாத்திரம் மாற்றத்தை ஏற்படுத்த முடியாது."

In [ ]:
X = transform_to_dataset(input_txt)

In [ ]:
X = [transform_to_dataset(input_txt)]

In [ ]:
type(X)

In [ ]:
print(X[0])

In [ ]:
model.predict(X)

In [ ]:
extract_features(["இல்­லையா"], 0)

----

In [ ]:
with open("./ner_crf_model.pkl", "rb") as f:
    ner_model = pickle.load(f)

In [ ]:
ner_model.classes_

In [ ]:
def extract_ner_features(sentence, index):
    word = sentence[index]
    return {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prefix-1': "" if not word else word[0],
        'prefix-2': word[:2],
        'prefix-3': word[:3],
        'prefix-4': word[:4],
        'suffix-1': "" if not word else word[-1],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
        'suffix-4': word[-4:],
        'prev_word': '' if index == 0 else sentence[index - 1][0],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1][0],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),
    }

In [ ]:
def transform_to_ner_dataset(corpus: str):
    sentences = sent_tokenize(corpus)
    sentences = [word_tokenize(sentence) for sentence in sentences]
    X = []
    for sentence in sentences:
        tmp = []
        for i in range(len(sentence)):
            tmp.append(extract_ner_features(sentence, i))
        X.append(tmp)
    return X

In [ ]:
input_txt

In [ ]:
input_txt += "இந்தியா,ஆஸ்திரேலியா, இலங்கை."

In [ ]:
X = transform_to_ner_dataset(input_txt)

In [ ]:
ner_model.predict(X)

---

In [ ]:
import pickle

In [ ]:
with open("./lstm_tokenizer.pkl", "rb") as f:
    lstm_tokenizer_model = pickle.load(f)

In [ ]:
lstm_tokenizer_model.texts_to_sequences(["mass thala ajith vijay rajini"])

In [ ]:
from keras_preprocessing.sequence import pad_sequences 

In [ ]:
tmp = pad_sequences(lstm_tokenizer_model.texts_to_sequences(["mass thala ajith vijay rajini"]))

In [ ]:
from keras.models import load_model

In [ ]:
lstm_model = load_model("./llm_lstm_model.keras/")

In [ ]:
lstm_model.predict(tmp)

In [ ]:
classes = ['Mixed_feelings','Negative','Positive','unknown_state']

In [ ]:
import numpy as np
pred_class = []
for item in lstm_model.predict(tmp):
  index = np.argmax(item)
  pred_class.append(classes[index])
  #pred_class.append(index)

print(pred_class)

In [ ]:
list(map(lambda x: classes[np.argmax(x)], lstm_model.predict(tmp)))

-----------

In [ ]:
import torch
import torch.nn as nn

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
class INTENT_CLASSIFIER(nn.Module):

  def __init__(self, freeze_bert=True):
    super(INTENT_CLASSIFIER, self).__init__()

    self.bert_layers = BertModel.from_pretrained('bert-base-multilingual-cased',return_dict=False)
    self.linear1 = nn.Linear(768, 300)
    self.linear11 = nn.Linear(300, 8)
    self.linear2 = nn.Linear(8, 2)
    self.dropout = nn.Dropout(0.5)

    if freeze_bert:
      for param in self.bert_layers.parameters():
        param.requires_grad = False


  def forward(self, token_ids, atten_mask):
    """Both argument are of shape: batch_size, max_seq_len"""
    _, CLS = self.bert_layers(token_ids, attention_mask = atten_mask)
    logits = self.dropout(self.linear1(CLS))
    logits = self.dropout(self.linear11(logits))
    logits = self.linear2(logits)

    return logits

In [ ]:
#loading the model
model = torch.load("./best_model.pth", map_location=torch.device('cpu'))

In [3]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import random
import unicodedata

/Users/deebakkarthi/.local/src/tanglishServer/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
test_file="./tamil_dev.csv"
test_set = ATIS(file_name = test_file, max_token_len=120, translit_prob=0, shuffle_prob=0)

In [ ]:
print(len(test_set))

In [ ]:
val_loader = DataLoader(test_set, batch_size = 64, num_workers =4, shuffle=False)

In [ ]:
# Getting the test accuracy
def test():
  correct = 0
  total = 0
  model.eval()
  y_test_prediction = []
  y_test_true = []

  with torch.no_grad():
      for data in val_loader:
          tokens, masks, labels = data

          tokens = tokens.to(device)
          masks = masks.to(device)
          labels = labels.to(device)

          outputs = model(tokens, masks)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels.squeeze()).sum().item()

          y_test_true += labels.squeeze().detach().cpu().numpy().tolist()
          y_test_prediction += predicted.detach().cpu().numpy().tolist()

  print('Test Accuracy: {}%'.format(100 * correct / total))
  test_accuracy_value = correct/total
  return y_test_true, y_test_prediction, test_accuracy_value

In [13]:
test_test_file="./tamil_test.csv"

In [14]:
from ATIS import ATIS

In [15]:
test_test_set = ATIS(file_name = test_test_file, max_token_len=120, translit_prob=0, shuffle_prob=0)

In [4]:
import re

In [5]:
test_test_set[9]

(tensor([  101, 12382, 12822, 12382, 10112, 66325, 33396,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,

In [11]:
from torch.utils.data import DataLoader

In [16]:
test_loader = DataLoader(test_test_set, batch_size = 1, num_workers =4, shuffle=False)

In [19]:
for i, data in enumerate(test_loader, 0):
  print(data[0].shape)
  print(data[1].shape)
  print(data[2])
  print(data[3])
  break

torch.Size([1, 120])
torch.Size([1, 120])
('yarayellam fdfs ppga ippove ready agitinga',)
('Positive',)


In [10]:
# path where model is saved and loaded
model_path = 'best_model.pth'

In [3]:
import torch

In [6]:
from INTENT_CLASSIFIER import INTENT_CLASSIFIER

/Users/deebakkarthi/.local/src/tanglishServer/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
#loading the model
model = torch.load("./best_model.pth", map_location=torch.device('cpu'))

NameError: name 'torch' is not defined

In [2]:
model

NameError: name 'model' is not defined

In [17]:
def test():
  correct = 0
  total = 0
  model.eval()
  y_test_prediction = []
  y_test_true = []

  with torch.no_grad():
      for data in test_loader:
          tokens, masks, labels = data

          tokens = tokens.to(device)
          masks = masks.to(device)
          labels = labels.to(device)

          outputs = model(tokens, masks)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels.squeeze()).sum().item()

          y_test_true += labels.squeeze().detach().cpu().numpy().tolist()
          y_test_prediction += predicted.detach().cpu().numpy().tolist()

  print('Test Accuracy: {}%'.format(100 * correct / total))
  test_accuracy_value = correct/total
  return y_test_true, y_test_prediction, test_accuracy_value

In [18]:
test()

ValueError: too many values to unpack (expected 3)

In [20]:
# Getting the test accuracy
def inference():
  model.eval()

  query_id_list = []
  query_list = []
  prediction_list = []
  running_sum = 0

  with torch.no_grad():
      for data in test_loader:
          tokens, masks, query, query_id = data

          query_id_list.append(query_id[0])
          query_list.append(query[0])

          tokens = tokens.to(device)
          masks = masks.to(device)

          outputs = model(tokens, masks)
          _, predicted = torch.max(outputs.data, 1)
          running_sum+=predicted.detach().cpu().item()

          prediction_list.append('off' if predicted==1 else 'not')
          # break

          # y_test_true += labels.squeeze().detach().cpu().numpy().tolist()
          # y_test_prediction += predicted.detach().cpu().numpy().tolist()

  # print('Test Accuracy: {}%'.format(100 * correct / total))
  # test_accuracy_value = correct/total
  return running_sum, query_id_list, query_list, prediction_list

In [22]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [23]:
inference()

In [9]:
model

INTENT_CLASSIFIER(
  (bert_layers): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, ele